## Get the polygons from the HydroPolys dataset that have been identified as a Reservoir. Now this code will take only the reservoirs that have names

In [1]:
%matplotlib inline

import sys

sys.path.append('Scripts')
from datahandling import wofs_fuser

import datacube
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
from datacube.utils import masking
from datacube.utils import geometry 
from datacube.utils.geometry import CRS
from datacube.helpers import write_geotiff
import geopandas as gpd
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json



dc = datacube.Datacube()

## There are over 1'000 reservoirs in this dataset but not all of them have names. 
They also don't have a lat, lon location which we'll need to match the gauges and call the satellite data, but that's ok we can get it. First have a look at the dataset I have. I went to the DEA website and downloaded the HydroPolys National dataset then in Arc, I filtered for the ones that were labelled as reservoirs.

In [ ]:
gdf = gpd.read_file('Reservoirs/Reservoirs.shp')
gdf

## Only take reservoirs with names and then get their centroid lat and lon

In [ ]:
gdf = gpd.read_file('Reservoirs/Reservoirs.shp')

gdf = gdf.to_crs(epsg=4326) #4326 is the tag for lat lon format
#we need to get the lat and lon of where the dam is to call the right satellite data later
#we want the lat and lon to be columns in the data to make it easy when we query the wofs data later
#Getting the centroid from the polygon and adding a latitude and longitude column (I found this code on the internet):
def getXY(pt):
    return (pt.x, pt.y)
centroidseries = gdf['geometry'].centroid
x,y = [list(t) for t in zip(*map(getXY, centroidseries))]

gdf = gdf.assign(lon=x)
gdf = gdf.assign(lat=y)
#gdf['info'] = list(zip(gdf.lat, gdf.lon, gdf.Area_calc))
gdf = gdf[['NAME', 'SHAPE_Leng', 'SHAPE_Area', 'Area_calc', 'lat', 'lon', 'geometry']]
gdf = gdf.dropna()
gdf

## Check it out on an interactive map (cool)

In [ ]:
#Converting the data to json
data = json.loads(gdf.to_json())

map = ipyl.Map(center=[-28, 148], zoom=7)

label = ipyw.Label(layout=ipyw.Layout(width='100%'))

for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.5
    }
layer = ipyl.GeoJSON(data=data, hover_style={'fillColor': 'red'})

def click_handler(event=None, feature=None, id=None, properties=None):
    label.value = str(properties['NAME'])
    
    
layer.on_hover(click_handler)
map.add_layer(layer)


ipyw.VBox([map, label])

## save out the named reservoirs shapefile with the centroid locations attached

In [ ]:
output = 'Named_Reservoirs'
gdf.to_file(output)

## I joined the gauges to these named reservoirs in ArcGIS with spatial join. Let's see:

In [2]:
gdf = gpd.read_file("Polys_with_gaugeID/pointsID_polys_join_1km.shp")
gdf

,Join_Count,TARGET_FID,JOIN_FID,NAME,SHAPE_Area,Area_calc,lat,lon,staion_nam,gauge_ID,lat_1,lon_1,geometry
0,1,0,257,WILSON RESERVOIR,0.000028,270561.736,-37.510018,144.018549,LAL LAL C WILSON RES,232220A,-37.514370,144.017660,"POLYGON ((144.01821 -37.51455, 144.01719 -37.5..."
1,0,1,-1,SUGARLOAF RESERVOIR,0.000438,4284887.270,-37.672015,145.305467,None,None,0.000000,0.000000,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
2,1,2,181,GREEN LAKE,0.000715,7111145.660,-36.435338,144.834187,Greens Lake,sp-o10454,-36.433798,144.834474,"POLYGON ((144.82094 -36.43662, 144.82081 -36.4..."
3,1,3,536,LAKE TAYLOR,0.000490,4853573.620,-36.783437,142.383614,Taylors Lake,TAYLORS,-36.773120,142.386560,"POLYGON ((142.39295 -36.77087, 142.39255 -36.7..."
4,1,4,554,UPPER STONY CREEK RESERVOIR,0.000051,500287.353,-37.818353,144.203635,Upper Stony,RE604,-37.823300,144.206200,"POLYGON ((144.21140 -37.81367, 144.21152 -37.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,1,669,333,MARIAN WEIR,0.000134,1544632.180,-21.146641,148.885758,Marian Weir HW,125003A,-21.141222,148.936264,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
818,1,669,342,MARIAN WEIR,0.000134,1544632.180,-21.146641,148.885758,Mirani Weir HW,125008A,-21.177880,148.830000,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
819,1,670,335,MARY BARRAGE,0.000115,1283147.610,-25.627152,152.613932,Mary River Barrage,138013B,-25.588400,152.625350,"POLYGON ((152.59687 -25.65457, 152.59709 -25.6..."
820,1,671,542,TINANA BARRAGE,0.000012,129235.966,-25.571684,152.713346,Tinana Barrage HW,138008A,-25.572060,152.718100,"POLYGON ((152.71974 -25.56929, 152.71973 -25.5..."


In [3]:
gdf = gdf.set_index(['gauge_ID'])
gdf

,Join_Count,TARGET_FID,JOIN_FID,NAME,SHAPE_Area,Area_calc,lat,lon,staion_nam,lat_1,lon_1,geometry
gauge_ID,,,,,,,,,,,,
232220A,1,0,257,WILSON RESERVOIR,0.000028,270561.736,-37.510018,144.018549,LAL LAL C WILSON RES,-37.514370,144.017660,"POLYGON ((144.01821 -37.51455, 144.01719 -37.5..."
NaN,0,1,-1,SUGARLOAF RESERVOIR,0.000438,4284887.270,-37.672015,145.305467,None,0.000000,0.000000,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
sp-o10454,1,2,181,GREEN LAKE,0.000715,7111145.660,-36.435338,144.834187,Greens Lake,-36.433798,144.834474,"POLYGON ((144.82094 -36.43662, 144.82081 -36.4..."
TAYLORS,1,3,536,LAKE TAYLOR,0.000490,4853573.620,-36.783437,142.383614,Taylors Lake,-36.773120,142.386560,"POLYGON ((142.39295 -36.77087, 142.39255 -36.7..."
RE604,1,4,554,UPPER STONY CREEK RESERVOIR,0.000051,500287.353,-37.818353,144.203635,Upper Stony,-37.823300,144.206200,"POLYGON ((144.21140 -37.81367, 144.21152 -37.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...
125003A,1,669,333,MARIAN WEIR,0.000134,1544632.180,-21.146641,148.885758,Marian Weir HW,-21.141222,148.936264,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
125008A,1,669,342,MARIAN WEIR,0.000134,1544632.180,-21.146641,148.885758,Mirani Weir HW,-21.177880,148.830000,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
138013B,1,670,335,MARY BARRAGE,0.000115,1283147.610,-25.627152,152.613932,Mary River Barrage,-25.588400,152.625350,"POLYGON ((152.59687 -25.65457, 152.59709 -25.6..."


In [23]:
ID = "136020A"
gdf1 = gdf.loc[ID]
gdf1

Join_Count                                                    1
TARGET_FID                                                  664
JOIN_FID                                                     50
NAME                                       BEN ANDERSON BARRAGE
SHAPE_Area                                          0.000538202
Area_calc                                           6.02193e+06
lat                                                    -24.9276
lon                                                     152.232
staion_nam                                 Ben Anderson Barrage
lat_1                                                    -24.93
lon_1                                                   152.255
geometry      POLYGON ((152.1670104999999 -24.99459350000001...
Name: 136020A, dtype: object

In [10]:
df = pd.DataFrame({Anderson}, {'Name'}, {'ID'})
df

TypeError: unhashable type: 'Polygon'